In [116]:
import requests, json, os, dotenv, openpyxl
import pandas as pd
import numpy as np
import pandasql as ps
from pandasql import sqldf

In [117]:
dotenv.load_dotenv('../ref/.env')

True

In [118]:
league_id = os.environ.get('tds')

In [7]:
def getNflState():
    nfl_url = 'https://api.sleeper.app/v1/state/nfl'
    r = requests.get(nfl_url)
    if r.status_code == 200:
        nfl_state = r.json()
        current_week = nfl_state['week']
        current_season = nfl_state['season']
        week_type = nfl_state['season_type']
        return current_week, current_season, week_type

In [8]:
def getLeagueInfo(league_id: str):
    league_url = f'https://api.sleeper.app/v1/league/{league_id}'
    r = requests.get(league_url)
    if r.status_code == 200:
        league_info = r.json()
        leagues = {}
        leagues[league_info['league_id']] = {
            'total_teams' : league_info['total_rosters'],
            'roster_poositions' : league_info['roster_positions'],
            'draft_id' : league_info['draft_id'],
            'season' : league_info['season'],
            'league_name' : league_info['name'],
            'playoff_start_wk' : league_info['settings']['playoff_week_start'],
            'num_playoff_teams' : league_info['settings']['playoff_teams'],
            'faab_budget' : league_info['settings']['waiver_budget']
            }
        return leagues
    else:
        return None

In [4]:
def getManagers(league_id: str):
    managers_url = f'https://api.sleeper.app/v1/league/{league_id}/users'
    managers_resp = requests.get(managers_url)
    if managers_resp.status_code == 200:
        managers_list = managers_resp.json()
        return pd.DataFrame([(manager['user_id'], manager['display_name']) for manager in managers_list], columns=['manager_id', 'display_name'])
    else:
        return None

In [5]:
managers_df = getManagers(league_id)

In [6]:
def getCurrentRosters(league_id: str):
    rosters_url = f'https://api.sleeper.app/v1/league/{league_id}/rosters'
    rosters_resp = requests.get(rosters_url)
    if rosters_resp.status_code == 200:
        rosters_list = rosters_resp.json()
        return pd.DataFrame([(roster['owner_id'], roster['players']) for roster in rosters_list], columns=['manager_id', 'players']).explode('players')
    else:
        return None


In [7]:
curr_rosters_df = getCurrentRosters(league_id)

In [14]:
def getRosters(league_id: str):
    rosters_url = f'https://api.sleeper.app/v1/league/{league_id}/rosters'
    r = requests.get(rosters_url)
    if r.status_code == 200:
        rosters_resp = r.json()
        rosters = []
        for roster in rosters_resp:
            settings = roster['settings']
            metadata = roster['metadata']
            owner_id = roster['owner_id']
            mngr_req = requests.get(f'https://api.sleeper.app/v1/user/{owner_id}')
            username = None
            if mngr_req.status_code == 200: username = mngr_req.json()['username']
            rosters.append([roster['roster_id'], owner_id, roster_manager[str(roster['roster_id'])], metadata['streak'], metadata['record'], settings['wins'], settings['losses'], settings['ties'], settings['waiver_budget_used']])
        rosters_df = pd.DataFrame(rosters, columns=['roster_id', 'owner_id', 'owner_name', 'current_streak', 'record', 'wins', 'losses', 'ties', 'faab_used'])
        rosters = {}
        for roster in rosters_resp:
            rosters[roster['owner_id']] = {
                'wins' : roster['settings']['wins'],
                'losses' : roster['settings']['losses'],
                'ties' : roster['settings']['ties'],
                'pts_for' : f'{roster['settings']['fpts']}.{roster['settings']['fpts_decimal']}',
                'pts_against' : f'{roster['settings']['fpts_against']}.{roster['settings']['fpts_against_decimal']}',
                'total_moves' : roster['settings']['total_moves'],
                'starters' : roster['starters'],
                'ir' : roster['reserve'],
                'bench' : list(set(roster['players']) - set(roster['starters'] if roster['starters'] is not None else []) - set(roster['reserve'] if roster['reserve'] is not None else []))
                }
        return rosters, rosters_df
    else:
        return None

In [15]:
def getTransactions(league_id: str, week: str):
    transaction_url = f'https://api.sleeper.app/v1/league/{league_id}/transactions/{week}'
    r = requests.get(transaction_url)
    statuses = []
    if r.status_code == 200:
        transactions = r.json()
        succesful_transactions = []
        for transaction in transactions:
            if transaction['status'] == 'complete':
                if transaction['type'] == 'waiver':
                    pass
                elif transaction['type'] == 'free_agent':
                    pass
                elif transaction['type'] == 'trade':
                    pass
                succesful_transactions.append(transaction)
        return succesful_transactions
    else:
        return None

In [16]:
def getMatchups(league_id: str, week: str):
    matchup_url = f'https://api.sleeper.app/v1/league/{league_id}/matchups/{week}'
    r=requests.get(matchup_url)
    matchup_list = []
    players_list = []
    if r.status_code == 200:
        matchups = r.json()
        for team in matchups:
            roster_id = team['roster_id']
            for player in team['players_points']:
                players_list.append([player, roster_id, int(week), True if player in team['starters'] else False, team['players_points'][player]])
            matchup_list.append([team['matchup_id'], int(week), roster_id])
    
    return players_list, matchup_list
        

In [11]:
draft_url = f'https://api.sleeper.app/v1/league/{league_id}/drafts'
draft_response=requests.get(draft_url)
if draft_response.status_code == 200:
    draft_dict = draft_response.json()[0]
    draft_id = draft_dict['draft_id']
    draft_picks_url = f'https://api.sleeper.app/v1/draft/{draft_id}/picks'
    draft_picks_resp = requests.get(draft_picks_url)
    if draft_picks_resp.status_code == 200:
        draft_picks = draft_picks_resp.json()

[{'draft_id': '1048281947404955649', 'draft_slot': 1, 'is_keeper': None, 'metadata': {'first_name': 'Kyren', 'injury_status': '', 'last_name': 'Williams', 'news_updated': '1724807459855', 'number': '23', 'player_id': '8150', 'position': 'RB', 'sport': 'nfl', 'status': 'Active', 'team': 'LAR', 'team_abbr': '', 'team_changed_at': '', 'years_exp': '2'}, 'pick_no': 1, 'picked_by': '988914274368020480', 'player_id': '8150', 'reactions': None, 'roster_id': 4, 'round': 1}, {'draft_id': '1048281947404955649', 'draft_slot': 2, 'is_keeper': None, 'metadata': {'first_name': 'Marvin', 'injury_status': '', 'last_name': 'Harrison', 'news_updated': '1724636137545', 'number': '18', 'player_id': '11628', 'position': 'WR', 'sport': 'nfl', 'status': 'Active', 'team': 'ARI', 'team_abbr': '', 'team_changed_at': '', 'years_exp': '0'}, 'pick_no': 2, 'picked_by': '992614081347579904', 'player_id': '11628', 'reactions': None, 'roster_id': 10, 'round': 1}, {'draft_id': '1048281947404955649', 'draft_slot': 3, 'i

In [13]:
draft_picks[1]

{'draft_id': '1048281947404955649',
 'draft_slot': 2,
 'is_keeper': None,
 'metadata': {'first_name': 'Marvin',
  'injury_status': '',
  'last_name': 'Harrison',
  'news_updated': '1724636137545',
  'number': '18',
  'player_id': '11628',
  'position': 'WR',
  'sport': 'nfl',
  'status': 'Active',
  'team': 'ARI',
  'team_abbr': '',
  'team_changed_at': '',
  'years_exp': '0'},
 'pick_no': 2,
 'picked_by': '992614081347579904',
 'player_id': '11628',
 'reactions': None,
 'roster_id': 10,
 'round': 1}

In [17]:
def getLeagueDraft(league_id: str):
    draft_url = f'https://api.sleeper.app/v1/league/{league_id}/drafts'
    draft_response=requests.get(draft_url)
    if draft_response.status_code == 200:
        draft_dict = draft_response.json()[0]
        draft_id = draft_dict['draft_id']
        draft_picks_url = f'https://api.sleeper.app/v1/draft/{draft_id}/picks'
        draft_picks_resp = requests.get(draft_picks_url)
        if draft_picks_resp.status_code == 200:
            draft_picks = draft_picks_resp.json()
            #picks = [(pick['player_id'], pick['round'], pick['picked_by'], pick['is_keeper'], int(pick['metadata']['amount'])) for pick in draft_picks]
            picks = [(pick['player_id'], pick['round'], pick['picked_by'], pick['is_keeper']) for pick in draft_picks]
            picks_df = pd.DataFrame(picks, columns=['player_id', 'round', 'manager_id', 'is_keeper'])
    return picks_df if not picks_df.empty else None

In [98]:
draft_picks_df = getLeagueDraft(league_id=league_id)

In [99]:
draft_picks_df['years_kept'] = draft_picks_df.apply(lambda x: 1 if x.is_keeper else 0, axis=1)

In [100]:
draft_picks_df.drop('is_keeper', axis=1, inplace=True)

In [26]:
prev_draft_round = {
    '7564' : 1,
    '3321' : 2,
    '4034' : 1,
    '2133' : 1,
    '2449' : 2,
    '6794' : 1,
    '6786' : 2,
    '4984' : 4,
    '8146' : 10,
    '7547' : 5,
    '4881' : 5,
    '4866' : 2
}

In [29]:
prev_keepers_df = pd.DataFrame(prev_draft_round.items(), columns=['player_id', 'round_drafted'])

In [31]:
prev_keepers_df['years_kept'] = 2

In [101]:
draft_picks_df = draft_picks_df.merge(prev_keepers_df, on=['player_id'], how='outer')

In [103]:
draft_picks_df = draft_picks_df.merge(managers_df)

In [105]:
draft_picks_df.to_excel('tds_keepers.xlsx')

In [ ]:
# TODO: add functionality to pull the highest scoring player available during the draft
def getBestDraftPlayer():
    pass

In [ ]:
# TODO: add playoff chance calculator

In [ ]:
# TODO: add clinching criteria, a la https://theffhub.com/PlayoffMachine?site=sleeper&leagueId=984533715101495296&userId=&s2=&swid=

In [ ]:
# TODO: add schedule comparison

In [ ]:
def longest_win_streak(row):
    streaks = row.split('L')
    max_win_streak = max(map(len, streaks))
    return max_win_streak

def longest_loss_streak(row):
    streaks = row.split('W')
    max_loss_streak = max(map(len, streaks))
    return max_loss_streak

In [119]:
def getPlayers(update_players=False):
    if update_players:
        print('Updating Players info...')
        players_url = 'https://api.sleeper.app/v1/players/nfl'
        players_resp = requests.get(players_url)
        if players_resp.status_code == 200:
            players_dict = players_resp.json()
            with open('../ref/players.json', 'w') as players_out:
                json.dump(players_dict, players_out, sort_keys=True, indent=4)
    with open('../ref/players.json') as players_in:
        players = json.load(players_in)
    players_list = []
    for player in players:
        position = players[player]['position']
        is_rookie = False
        if position != 'DEF':
            is_rookie = True if players[player]['years_exp'] == 0 else False
        if players[player]['active']: players_list.append([player, players[player]['first_name'], players[player]['last_name'], is_rookie, position])
    return pd.DataFrame(players_list, columns=['player_id', 'first_name', 'last_name', 'is_rookie', 'position'])


In [120]:
players_df = getPlayers()

In [121]:
keepers_df = pd.read_excel('tds_keepers.xlsx')

In [123]:
keepers_df = keepers_df.merge(players_df)

In [126]:
keepers_df.drop(['player_id', 'is_rookie'], axis=1, inplace=True)

In [128]:
keepers_df.fillna(0, inplace=True)

In [129]:
keepers_df.to_excel('tds_keepers.xlsx')

In [36]:
draft_picks_df['years_kept'] = 1
draft_picks_df['base_salary'] = round(draft_picks_df.cost * (1 + (draft_picks_df.years_kept * 0.2)))

In [43]:
rosterd_players_df = sqldf("""
SELECT
    players_df.player_id
    ,players_df.first_name
    ,players_df.last_name
    ,players_df.position
    ,Coalesce(draft_picks_df.base_salary, 5) AS base_salary
    ,Coalesce(draft_picks_df.years_kept,0) AS years_kept
    ,managers_df.manager_id
    ,managers_df.display_name
    FROM players_df
    LEFT JOIN curr_rosters_df
        ON curr_rosters_df.players = players_df.player_id
    LEFT JOIN managers_df
        ON managers_df.manager_id = curr_rosters_df.manager_id
    LEFT JOIN draft_picks_df
        ON draft_picks_df.player_id = players_df.player_id
    WHERE players_df.player_id IN (SELECT players FROM curr_rosters_df)
""")

In [44]:
rosterd_players_df

,player_id,first_name,last_name,position,base_salary,years_kept,manager_id,display_name
0,11533,Brandon,Aubrey,K,5.0,1,862460871958462464,TravisBailey
1,6786,CeeDee,Lamb,WR,77.0,1,862460871958462464,TravisBailey
2,6783,Jerry,Jeudy,WR,1.0,1,1004853775678500864,jdblackwell21
3,11439,Jaleel,McLaughlin,RB,1.0,1,79241179040202752,Sunshine81
4,7553,Kyle,Pitts,TE,20.0,1,735752104337092608,Drunkworth
...,...,...,...,...,...,...,...,...
176,8155,Breece,Hall,RB,77.0,1,870168774161178624,kerrveball
177,KC,Kansas City,Chiefs,DEF,1.0,1,855933294594879488,rmasons
178,6806,J.K.,Dobbins,RB,1.0,1,862460871958462464,TravisBailey
179,6813,Jonathan,Taylor,RB,55.0,1,988915873790459904,GriffDaGreat


In [ ]:
rosterd_players_df.to_excel('tds_2023.xlsx', index=False)

In [ ]:

current_week, current_season, nfl_state = getNflState()
transactions = {}
player_list = []
matchup_list = []
for wk in range(1, current_week):
    transactions[str(wk)] = getTransactions(league_id, str(wk))
    player_week_list, matchup_week_list = getMatchups(league_id, str(wk))
    player_list.extend(player_week_list)
    matchup_list.extend(matchup_week_list)


In [ ]:
league_info = getLeagueInfo(league_id=league_id)

In [ ]:
league_info

In [ ]:
rosters, rosters_df = getCurrentRosters(league_id)

In [ ]:
rosters_df['longest_win_streak'] = rosters_df['record'].apply(longest_win_streak)
rosters_df['longest_loss_streak'] = rosters_df['record'].apply(longest_loss_streak)

In [ ]:
player_info_df = getPlayers()

In [ ]:
player_df = pd.DataFrame(player_list, columns=['player_id', 'roster_id', 'week', 'is_starter', 'points'])
matchup_df = pd.DataFrame(matchup_list, columns=['matchup_id', 'week', 'roster_id'])

In [ ]:
player_df = ps.sqldf("""
SELECT
    pdf.*
    ,pi.first_name
    ,pi.last_name
    ,pi.is_rookie
    ,pi.position
    FROM player_df pdf
    LEFT JOIN player_info_df pi
        ON pi.player_id = pdf.player_id          
""")

In [ ]:
weekly_pos_rnks = ps.sqldf("""
SELECT
    roster_id
    ,week
    ,player_id
    ,position
    ,points
    ,Rank() OVER (PARTITION BY roster_id, week, position ORDER BY points DESC, player_id) AS pos_rnk
    FROM player_df
    WHERE 1=1
""")

In [ ]:
top_weekly_players = ps.sqldf("""
SELECT
    roster_id
    ,week
    ,player_id
    FROM
        (SELECT
            roster_id
            ,week
            ,player_id
            ,position
            ,points
            ,Rank() OVER (PARTITION BY roster_id, week ORDER BY points DESC, player_id) AS flex_rnk
            FROM weekly_pos_rnks
            WHERE 1=1
                AND ((position IN ('WR', 'RB') AND pos_rnk > 2) OR (position = 'TE' AND pos_rnk > 1))   
        )
    WHERE flex_rnk < 3
UNION ALL
SELECT
    roster_id
    ,week
    ,player_id
    FROM weekly_pos_rnks
    WHERE 1=1
        AND ((position NOT IN ('WR', 'RB') AND pos_rnk = 1) OR (position IN ('WR', 'RB') AND pos_rnk < 3))
""")

In [ ]:
player_df = ps.sqldf("""
SELECT
    fp.*
    ,CASE
        WHEN tp.player_id IS NOT NULL
            THEN True
        ELSE False
    END AS top_player
    FROM player_df fp
    LEFT JOIN top_weekly_players tp
        ON tp.player_id = fp.player_id
        AND tp.week = fp.week
        AND tp.roster_id = fp.roster_id
""")

In [ ]:
starters_by_score = """
SELECT
    player_id
    ,first_name
    ,last_name
    ,Sum(points) AS ttl_strtd_points
    FROM player_df
    WHERE 1=1
        AND is_starter = 1
    GROUP BY 1,2,3
    ORDER BY 4 DESC
"""

In [ ]:
ps.sqldf(starters_by_score).head(25)

In [ ]:
player_df

In [ ]:
rookies_by_manager_sql = """
SELECT
    rm.owner_name
    ,count(distinct player_id) AS rookies_started
    FROM player_df pdf
    LEFT JOIN roster_manager_df rm
        ON rm.roster_id = pdf.roster_id
    WHERE 1=1
        AND is_rookie = 1
        AND is_starter = 1
    GROUP BY 1
    ORDER BY rookies_started DESC
"""

In [ ]:
ps.sqldf(rookies_by_manager_sql)

In [ ]:
manager_yearly_points = ps.sqldf("""
SELECT
    roster_id
    ,Sum(CASE
        WHEN is_Starter = 1
            THEN points
        ELSE 0
    END) AS starter_points
    ,Sum(CASE
        WHEN top_player = 1
            THEN points
        ELSE 0
    END) AS top_points
    FROM player_df
    GROUP BY 1
""")

In [ ]:
mngr_effcncy_sql = """
SELECT
    rm.owner_name
    ,(starter_points/top_points) * 100 AS effcncy
    FROM manager_yearly_points myp
    LEFT JOIN roster_manager_df rm
        ON rm.roster_id = myp.roster_id
    ORDER BY effcncy DESC
"""

In [ ]:
ps.sqldf(mngr_effcncy_sql)

In [ ]:
reg_season_avg_scores_sql = """
SELECT
    rm.owner_name
    ,Sum(pdf.points) as ttl_points
    FROM player_df pdf
    LEFT JOIN roster_manager_df rm
        ON rm.roster_id = pdf.roster_id
    WHERE 1=1
        AND pdf.is_starter = 1
        AND pdf.week < 15
    GROUP BY 1
    ORDER BY 2 DESC
"""

In [ ]:
ps.sqldf(reg_season_avg_scores_sql)

In [ ]:
mngr_wkly_pnts = """
SELECT
    roster_id
    ,week
    ,sum(points) AS ttl_points
    FROM player_df
    WHERE 1=1
        AND is_starter =1 
    GROUP BY 1,2
"""

In [ ]:
mngr_wkly = ps.sqldf(mngr_wkly_pnts)

In [ ]:
mtchps_sql = """
SELECT
    a.week
    ,CASE
        WHEN a.roster_id < b.roster_id
            THEN a.roster_id
        ELSE b.roster_id
    END AS mngr_a
    ,CASE
        WHEN a.roster_id > b.roster_id
            THEN a.roster_id
        ELSE b.roster_id
    END AS mngr_b
    FROM matchup_df a
    LEFT JOIN matchup_df b
        ON b.matchup_id = a.matchup_id
        AND b.roster_id != a.roster_id
        AND b.week = a.week
    WHERE 1=1
        AND b.roster_id IS NOT NULL
    GROUP BY 1,2,3
"""

In [ ]:
mtchups_df = ps.sqldf(mtchps_sql)

In [ ]:
mtchp_pnts_sql = """
SELECT
    m.week
    ,rma.owner_name AS owner_a
    ,a.ttl_points AS mngr_a_points
    ,rmb.owner_name AS owner_b
    ,b.ttl_points AS mngr_b_points
    ,CASE
        WHEN a.ttl_points > b.ttl_points
            THEN rma.owner_name
        ELSE rmb.owner_name
    END AS winner
    ,CASE
        WHEN a.ttl_points > b.ttl_points
            THEN a.ttl_points - b.ttl_points
        ELSE b.ttl_points - a.ttl_points
    END AS pnt_diff
    FROM mtchups_df m
    LEFT JOIN mngr_wkly a
        ON a.roster_id = m.mngr_a
        AND a.week = m.week
    LEFT JOIN mngr_wkly b
        ON b.roster_id = m.mngr_b
        AND b.week = m.week
    LEFT JOIN roster_manager_df rma
        ON rma.roster_id = m.mngr_a
    LEFT JOIN roster_manager_df rmb
        ON rmb.roster_id = m.mngr_b
    ORDER BY 7 DESC
"""

In [ ]:
matchup_ttls_df = ps.sqldf(mtchp_pnts_sql)

In [ ]:
ps.sqldf('SELECT * FROM matchup_ttls_df WHERE mngr_b_points < 111.62 OR mngr_a_points < 111.62')

In [ ]:
matchup_ttls_df

In [ ]:
lrgst_mrgn_vctry = """
SELECT
    winner
    ,sum(pnt_diff) / count(week) AS avg_vctry
    FROM matchup_ttls_df
    GROUP BY 1
"""

In [ ]:
ps.sqldf(lrgst_mrgn_vctry)

In [ ]:
ps.sqldf(mngr_wkly_pnts)

In [113]:
ttl = 0
for i in range(0,14):
    ttl += 5

In [114]:
ttl

70

In [115]:
for i in range(0,14):
    print(70 - (5 * (i)))

70
65
60
55
50
45
40
35
30
25
20
15
10
5
